# 尝试路由不同的数据源

使用路由查询引擎，对不同数据源的选择查询。

初步结论：

- 是可以的


## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size = 128
Settings.chunk_overlap = 10

CPU times: user 3.41 s, sys: 441 ms, total: 3.85 s
Wall time: 3.5 s


## 加载数据

In [3]:
%%time

from llama_index.readers.web import TrafilaturaWebReader

documents_yiheyuan = TrafilaturaWebReader().load_data(
    [ "https://baike.baidu.com/item/颐和园" for item in items]
)

documents_beihai = TrafilaturaWebReader().load_data(
    [ "https://baike.baidu.com/item/北海公园" for item in items]
)

CPU times: user 332 ms, sys: 76 µs, total: 332 ms
Wall time: 499 ms


## 建立索引

In [4]:
%%time

from llama_index.core import VectorStoreIndex

nodes = Settings.node_parser.get_nodes_from_documents(documents_yiheyuan)
vector_yiheyuan = VectorStoreIndex(nodes)

nodes = Settings.node_parser.get_nodes_from_documents(documents_beihai)
vector_beihai = VectorStoreIndex(nodes)

CPU times: user 1.5 s, sys: 72.3 ms, total: 1.57 s
Wall time: 32.7 s


## 建立查询引擎

In [6]:
%%time

from llama_index.core.tools import QueryEngineTool

yiheyuan_tool = QueryEngineTool.from_defaults(
    query_engine=vector_yiheyuan.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    ),
    description=(
        "适用于颐和园相关问题。"
    ),
)

beihai_tool = QueryEngineTool.from_defaults(
    query_engine=vector_beihai.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    ),
    description=(
        "适用于北海公园相关问题。"
    ),
)

CPU times: user 207 µs, sys: 0 ns, total: 207 µs
Wall time: 209 µs


In [7]:
%%time

from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        yiheyuan_tool,
        beihai_tool,
    ],
)

CPU times: user 216 µs, sys: 0 ns, total: 216 µs
Wall time: 219 µs


## 查询

In [8]:
%%time

response = query_engine.query("介绍下颐和园")
print(str(response))

颐和园是中国清朝时期的皇家园林，位于北京市西郊海淀区新建宫门路19号。它前身为清漪园，始建于清乾隆十五年（1750年），在咸丰十年（1860年）被英法联军烧毁后得以重建。颐和园利用昆明湖、万寿山为基址，并借鉴了杭州西湖的风景作为设计蓝本，融合了江南园林的设计手法和意境。它不仅是保存最完整的皇家行宫御苑，还被誉为“皇家园林博物馆”。全园占地总面积约为3.0平方千米。
CPU times: user 77 ms, sys: 0 ns, total: 77 ms
Wall time: 4.91 s


In [9]:
%%time

response = query_engine.query("夏天颐和园的门票信息")
print(str(response))

夏天颐和园的门票价格为30元/张。
CPU times: user 25.4 ms, sys: 68 µs, total: 25.5 ms
Wall time: 1.37 s


In [10]:
%%time

response = query_engine.query("北海公园在哪里，怎么坐车")
print(str(response))

北海公园位于中国北京市西城区文津街1号。要到达北海公园，您可以选择多种交通方式：

1. **公共交通**：
   - 地铁：乘坐地铁至南池子站或南池子北站（6号线），然后步行约500米即可到达。
   - 公交车：有多条公交线路经过北海公园附近，如1、2、8、13、47、107、111、118、120、122、609等。您可以在附近的站点下车后步行前往。

2. **打车或自驾**：
   - 使用手机地图应用（如高德地图、百度地图）输入“北海公园”，即可获取从您当前位置到北海公园的路线指引和实时交通信息。
   - 如果选择自驾车，建议使用导航系统以获得最准确的道路指引。

3. **步行**：
   - 北海公园周边有多个景点，如果您在北京市内，可以选择步行前往。例如，东邻景山公园，南濒中南海，北连什刹海，这些地方都相对靠近，您可以根据自己的行程规划选择合适的路线。

到达北海公园后，请遵守园区规定和安全提示，享受您的游览时光。
CPU times: user 19.3 ms, sys: 263 µs, total: 19.6 ms
Wall time: 6 s
